#EXTRACTING DATA

In [1]:
# IMPORTING PACKAGES

import pandas as pd
import requests
import pandas_ta as ta
#import matplotlib.pyplot as plt
#from termcolor import colored as cl
#import math 
from datetime import datetime, timedelta
import sqlite3

#plt.rcParams['figure.figsize'] = (20,10)
#plt.style.use('fivethirtyeight')

## Getting historical data from binance API

In [2]:
#The variable days_back represents the number of days in the past from which I intend to retrieve the data.
#The variable symbol is a coin pair. For example if you want bitcoin historical data, the symbol to use is BTCUSDT. 

API_URL = 'https://api.binance.com/api/v3/klines' # Binance API endpoint for candlestick data

def get_historical_data(symbol, days_back, interval):
    # Calculate the start time in milliseconds
    start_time = int((datetime.now() - timedelta(days=days_back)).timestamp() * 1000)

    # Calculate the current time in milliseconds
    end_time = int(datetime.now().timestamp() * 1000)

    # Make an API request to retrieve historical data
    hist_json = requests.get(API_URL, params={
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'endTime': end_time,
        'limit': 1000  # Maximum number of data points per request
    }).json()

    #print(hist_json)

    # Making a df from response
    # Extract only the relevant elements for each row
    df = pd.DataFrame(hist_json, columns=[
        'Time', 'Open Price', 'High Price', 'Low Price',
        'Close Price', 'Volume', 'Kline Close Time', 'Quote Asset Volume',
        'Number of Trades', 'Taker Buy Base Asset Volume',
        'Taker Buy Quote Asset Volume', 'Unused Field'
    ])

    # Select only the relevant columns (Open Time, Open Price, High Price, Low Price, Close Price, Volume)
    df = df[['Time', 'Open Price', 'High Price', 'Low Price', 'Close Price', 'Volume']]
    df['dateTime'] = pd.to_datetime(df['Time'], unit='ms')
    
    return df


## creating a table for calculating elder impulse

In [ ]:
Elder Impulse
https://school.stockcharts.com/doku.php?id=chart_analysis:elder_impulse_system
MACD-histogram
https://www.investopedia.com/ask/answers/122414/what-moving-average-convergence-divergence-macd-formula-and-how-it-calculated.asp

In [19]:
df_elder = get_historical_data('BTCUSDT', 30, '1d')


In [20]:
df_elder.tail()


,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime
25,1707436800000,45288.66000000,48200.00000000,45242.12000000,47132.77000000,73503.48100000,2024-02-09
26,1707523200000,47132.78000000,48170.00000000,46800.00000000,47751.09000000,24802.35936000,2024-02-10
27,1707609600000,47751.08000000,48592.66000000,47557.16000000,48299.99000000,29958.80837000,2024-02-11
28,1707696000000,48300.00000000,50334.82000000,47710.01000000,49917.27000000,59009.96705000,2024-02-12
29,1707782400000,49917.28000000,50368.61000000,48300.95000000,49163.78000000,47913.61831000,2024-02-13


In [21]:
df_elder['13EMA'] = df_elder['Close Price'].ewm(span=13, adjust=False).mean()

In [22]:
df_elder.tail()

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,13EMA
25,1707436800000,45288.66000000,48200.00000000,45242.12000000,47132.77000000,73503.48100000,2024-02-09,43731.059158
26,1707523200000,47132.78000000,48170.00000000,46800.00000000,47751.09000000,24802.35936000,2024-02-10,44305.349278
27,1707609600000,47751.08000000,48592.66000000,47557.16000000,48299.99000000,29958.80837000,2024-02-11,44876.012239
28,1707696000000,48300.00000000,50334.82000000,47710.01000000,49917.27000000,59009.96705000,2024-02-12,45596.191919
29,1707782400000,49917.28000000,50368.61000000,48300.95000000,49163.78000000,47913.61831000,2024-02-13,46105.847359


In [23]:
# Calculate 12-period EMA
df_elder['12EMA'] = df_elder['Close Price'].ewm(span=12, adjust=False).mean()

# Calculate 26-period EMA
df_elder['26EMA'] = df_elder['Close Price'].ewm(span=26, adjust=False).mean()

# Calculate MACD Line (12EMA - 26EMA)
df_elder['MACD'] = df_elder['12EMA'] - df_elder['26EMA']

# Calculate Signal Line (9-period EMA of MACD)
df_elder['Signal'] = df_elder['MACD'].ewm(span=9, adjust=False).mean()

# Calculate MACD Histogram
df_elder['Histogram'] = df_elder['MACD'] - df_elder['Signal']

In [24]:
df_elder.tail()

,Time,Open Price,High Price,Low Price,Close Price,Volume,dateTime,13EMA,12EMA,26EMA,MACD,Signal,Histogram
25,1707436800000,45288.66000000,48200.00000000,45242.12000000,47132.77000000,73503.48100000,2024-02-09,43731.059158,43832.816377,43020.198550,812.617826,350.569135,462.048691
26,1707523200000,47132.78000000,48170.00000000,46800.00000000,47751.09000000,24802.35936000,2024-02-10,44305.349278,44435.627703,43370.634954,1064.992749,493.453858,571.538891
27,1707609600000,47751.08000000,48592.66000000,47557.16000000,48299.99000000,29958.80837000,2024-02-11,44876.012239,45030.144980,43735.772365,1294.372615,653.637610,640.735005
28,1707696000000,48300.00000000,50334.82000000,47710.01000000,49917.27000000,59009.96705000,2024-02-12,45596.191919,45782.010368,44193.661079,1588.349289,840.579945,747.769343
29,1707782400000,49917.28000000,50368.61000000,48300.95000000,49163.78000000,47913.61831000,2024-02-13,46105.847359,46302.282619,44561.818036,1740.464583,1020.556873,719.907710


## Calculating elder impulse

In [30]:
last_row_13ema = df_elder.iloc[-1]['13EMA']
second_to_last_row_13ema = df_elder.iloc[-2]['13EMA']
# Print the comparison
print(f"Last row EMA_13: {last_row_13ema}")
print(f"Second-to-last row EMA_13: {second_to_last_row_13ema}")

Last row EMA_13: 46105.84735904187
Second-to-last row EMA_13: 45596.191918882185


In [31]:
ema_comparison = 0

if last_row_13ema > second_to_last_row_13ema:
    ema_comparison = 1
elif last_row_13ema < second_to_last_row_13ema:
    ema_comparison = -1

ema_comparison

1

In [34]:
elder_impulse = 0

if ema_comparison == 1 and df_elder['Histogram'].iloc[-1] > 0:
    elder_impulse = 1
elif ema_comparison == -1 and df_elder['Histogram'].iloc[-1] < 0:
    elder_impulse = -1
else:
    elder_impulse = 0

elder_impulse

1

if elder_impulse is 1, it means you can go long; 
if it is -1 , you can go short;
if it is 0 you can do nothing